# Calculating $R_0$ and $R_{eff}$

In [ ]:
from julia import Julia
jl = Julia(sysimage = "/home/callum/ASF/Fitting/sys_model.so") #loading sys image

In [ ]:
import matplotlib.pyplot as plt

import os
import tempfile
import numpy as np
import scipy as sp

from scipy import stats
import random as rd
from brokenaxes import brokenaxes

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [ ]:
plt.style.use('seaborn-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
#reading the fitted posteriors that we will run the model from
def read_inputs(path,number):
    
    models = [1,2,3,4]
    
    if number not in models:
        raise Exception("Model number must be 1, 2, 3 or 4") 
        
    weights = np.genfromtxt(path + "/H{}_weights.csv".format(number), delimiter=',')
    params = np.genfromtxt(path + "/H{}_params.csv".format(number), delimiter=',')
    
    nparams = params.shape[1]
    
    p1_dis = stats.gaussian_kde(params[:,0],weights = weights)
    p2_dis = stats.gaussian_kde(params[:,1],weights = weights)
    
    if nparams == 2:
        return p1_dis, p2_dis
    elif nparams == 3:
        p3_dis = stats.gaussian_kde(params[:,2],weights = weights)
        return p1_dis, p2_dis, p3_dis
    

In [ ]:
#simulating the model from the posteriors

def run_simulations_inter(path,number,n_sims, value, median = False):
    
    posteriors = read_inputs(path,number)
    
    n_params = len(posteriors) #two or three fitted parameters!
    
    dd = 0
    n_samples = 100000
    
    dp1 = posteriors[0].resample(n_samples)[0]
    dp2 = posteriors[1].resample(n_samples)[0]

    rd.shuffle(dp1)
    rd.shuffle(dp2)
    
    if n_params == 3:
        dp3 = posteriors[2].resample(n_samples)[0]
        rd.shuffle(dp3)
        
    if median:
        if n_params == 2:
            p = (np.median(dp1), np.median(dp2), value,number)
            
        elif n_params ==3:
             p = (np.median(dp1), np.median(dp2), np.median(dp3), value,number)
            
    for i in range(n_sims):
        
        if not median:
            if n_params == 2:
                p = (dp1[i], dp2[i], value,number)
            elif n_params ==3:
                p = (dp1[i], dp2[i], dp3[i], value,number)

        
        
        out = model_int(p)
        
        
        #dd += out
  
    return out[0], out[1], dp1[0], dp2[0]


# ODE model

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/ODE_Extinct.jl")')
jl.include('/home/callum/ASF/Fitting/ODE_Extinct.jl') #loading files with our model!

#Loading the four models!
model_int = jl.SIR_ODE.model_int #Frequency


In [ ]:
def fd(N):
    #calculating the net death rate of model!
    
    dr = μ*(σ + ((1-σ))*np.sqrt(N/K))
    
    #This section is to ensure the right output type occurs to ensure all multiplication works!
    if type(N) == float:
        return dr
    else:
        return np.expand_dims(dr,axis=1)

In [ ]:
#Key paparms
t = np.arange(1096) #time
λ = 1 / (60 + 30*np.cos((2*np.pi*(t+182.5))/365)) #corpse decay rate
σ = 0.75 #density independent ratio
K = 5000 #carrying capacity
μ = 0.0036 #naive death rate (daily)
ρ = 0.95 #mortatily 
γ =  0.125 #infection rate
ζ = 1/6 #exposed rate

In [ ]:
def simulate_r0_data(n_sims,model_no):
    
    #arrays to store out
    s_pop = np.zeros((n_sims,3*365+1)) #S
    n_pop = np.zeros((n_sims,3*365+1)) #N

    bh = np.zeros(n_sims) #beta
    o = np.zeros(n_sims) #omega
    
    for i in range(n_sims):
        s_pop[i,:], n_pop[i,:], bh[i], o[i] = run_simulations_inter(path,model_no,1,1) #(1 sim, 1 intevention (akin to nothing))
    
    return s_pop, n_pop, bh, o


In [ ]:
path = "Posteriors/ODE/"

In [ ]:
s_pop_1, n_pop_1, bh_1, o_1 = simulate_r0_data(500,1)
s_pop_2, n_pop_2, bh_2, o_2 = simulate_r0_data(500,2)
s_pop_3, n_pop_3, bh_3, o_3 = simulate_r0_data(500,3)
s_pop_4, n_pop_4, bh_4, o_4 = simulate_r0_data(500,4)


# TAU-Homogeneous model!

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HOMO_EXTINCT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HOMO_EXTINCT.jl') #loading files with our model!

#Loading the four models!
model_int = jl.SIR_TAU_S.model_int #Frequency


In [ ]:
path = "Posteriors/Tau-Homogeneous/"

In [ ]:
s_pop_1t, n_pop_1t, bh_1t, o_1t = simulate_r0_data(500,1)
s_pop_2t, n_pop_2t, bh_2t, o_2t = simulate_r0_data(500,2)
s_pop_3t, n_pop_3t, bh_3t, o_3t = simulate_r0_data(500,3)
s_pop_4t, n_pop_4t, bh_4t, o_4t = simulate_r0_data(500,4)

## Plotting Results!

In [ ]:
#Fixing the dimensions for calulations!
bh1 = np.expand_dims(bh_1,axis=1)
bh2 = np.expand_dims(bh_2,axis=1)
bh3 = np.expand_dims(bh_3,axis=1)
bh4 = np.expand_dims(bh_4,axis=1)

o1 = np.expand_dims(o_1,axis=1)
o2 = np.expand_dims(o_2,axis=1)
o3 = np.expand_dims(o_3,axis=1)
o4 = np.expand_dims(o_4,axis=1)

In [ ]:
bh1t = np.expand_dims(bh_1t,axis=1)
bh2t = np.expand_dims(bh_2t,axis=1)
bh3t = np.expand_dims(bh_3t,axis=1)
bh4t = np.expand_dims(bh_4t,axis=1)

o1t = np.expand_dims(o_1t,axis=1)
o2t = np.expand_dims(o_2t,axis=1)
o3t = np.expand_dims(o_3t,axis=1)
o4t = np.expand_dims(o_4t,axis=1)

In [ ]:
#Calculating R0!

R01i =  bh1*ζ*(λ+o1*(fd(n_pop_1)+ρ*γ))/(λ*(fd(n_pop_1)+γ)*(fd(n_pop_1)+ζ))

R02i =  (n_pop_2/K)*bh2*ζ*(λ+o2*(fd(n_pop_2)+ρ*γ))/(λ*(fd(n_pop_2)+γ)*(fd(n_pop_2)+ζ))

R03i = (np.tanh(1.5 *n_pop_3/K - 1.5 ) + 1)*bh3*ζ*(λ+o3*(fd(n_pop_3)+ρ*γ))/(λ*(fd(n_pop_3)+γ)*(fd(n_pop_3)+ζ))

R04i =  bh4*np.sqrt(n_pop_4/K)*ζ*(λ+o4*(fd(n_pop_4)+ρ*γ))/(λ*(fd(n_pop_4)+γ)*(fd(n_pop_4)+ζ))


In [ ]:
R01it =  bh1t*ζ*(λ+o1t*(fd(n_pop_1t)+ρ*γ))/(λ*(fd(n_pop_1t)+γ)*(fd(n_pop_1t)+ζ))

R02it =  (n_pop_2t/K)*bh2t*ζ*(λ+o2t*(fd(n_pop_2t)+ρ*γ))/(λ*(fd(n_pop_2t)+γ)*(fd(n_pop_2t)+ζ))

R03it = (np.tanh(1.5 *n_pop_3t/K - 1.5 ) + 1)*bh3t*ζ*(λ+o3t*(fd(n_pop_3t)+ρ*γ))/(λ*(fd(n_pop_3t)+γ)*(fd(n_pop_3t)+ζ))

R04it =  bh4t*np.sqrt(n_pop_4t/K)*ζ*(λ+o4t*(fd(n_pop_4t)+ρ*γ))/(λ*(fd(n_pop_4t)+γ)*(fd(n_pop_4t)+ζ))


In [ ]:
#Plotting!

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (4,4))
fig.tight_layout() # Or equivalently,  "plt.tight_layout()"
fig.suptitle('Reproduction Number',y=1.07, x = 0.52)
plt.subplots_adjust(hspace=0.4)

ax1.plot(t,np.mean(R01i, axis=0), color=colors[0], label = r"$C = 1$")
ax1.plot(t,np.mean(R01i*s_pop_1/n_pop_1, axis = 0), color=colors[0], ls = "--", label = r"$C = 1$")


ax1.plot(t,np.mean(R02i, axis=0), color=colors[1], label = r"$C \propto \rho_N: R_0$")
ax1.plot(t,np.mean(R02i*s_pop_2/n_pop_2, axis = 0), color=colors[1], ls = "--", label = r"$C \propto \rho_N$")

ax1.plot(t,np.mean(R03i, axis=0), color=colors[2], label = r"$C \propto \tanh(\rho_N): R_0$")
ax1.plot(t,np.mean(R03i*s_pop_3/n_pop_3, axis = 0), color=colors[2], ls = "--", label = r"$C \propto \tanh(\rho_N): R_{eff}$")



ax1.plot(t,np.mean(R04i, axis=0), color=colors[3], label = r"$C \propto \sqrt {\rho_N}: R_0$")
ax1.plot(t,np.mean(R04i*s_pop_4/n_pop_4, axis = 0), color=colors[3], ls = "--", label = r"$C \propto \sqrt {\rho_N}: R_{eff}$")


ax1.set_ylabel("Reproduction Number")



ax2.plot(t,np.mean(R01it, axis=0), color=colors[0], label = r"$R_0:\quad \, C = 1$")
ax2.plot(t,np.mean(R01it*s_pop_1t/n_pop_1t, axis = 0), color=colors[0], ls = "--", label = r"$R_{eff}: C = 1$")


ax2.plot(t,np.mean(R02it, axis=0), color=colors[1], label = r"$R_0: \quad \,  C \propto \rho_N$")
ax2.plot(t,np.mean(R02it*s_pop_2t/n_pop_2t, axis = 0), color=colors[1], ls = "--", label = r"$R_{eff}: C \propto \rho_N$")

ax2.plot(t,np.mean(R03it, axis=0), color=colors[2], label = r"$R_0: \quad \,  C  \propto \tanh(\rho_N)$")
ax2.plot(t,np.mean(R03it*s_pop_3t/n_pop_3t, axis = 0), color=colors[2], ls = "--", label = r"$R_{eff}: C \propto \tanh(\rho_N)$")



ax2.plot(t,np.mean(R04it, axis=0), color=colors[3], label = r"$R_0: \quad \, C \propto \sqrt {\rho_N}$")
ax2.plot(t,np.mean(R04it*s_pop_4t/n_pop_4t, axis = 0), color=colors[3], ls = "--", label = r"$R_{eff}: C \propto \sqrt {\rho_N}$")

ax2.legend(fontsize=7, ncol=2)


ax1.set_title("ODE")
ax2.set_title("Tau-Leaping Homogeneous")

ax2.set_xlabel("Day")
ax2.set_ylabel("Reproduction Number")
ax2.set_yscale('log')
ax1.set_yscale('log')
plt.savefig('m1r.pdf', format='pdf', bbox_inches='tight')
